In [ ]:
import pathlib

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import modelingutils as mu

%config Completer.use_jedi = False

From https://en.wikipedia.org/wiki/Knapsack_problem (accessed 2/10/2021):

> The knapsack problem is a problem in combinatorial optimization: Given a set of items, each with a weight and a value, determine the number of each item to include in a collection so that the total weight is less than or equal to a given limit and the total value is as large as possible. It derives its name from the problem faced by someone who is constrained by a fixed-size knapsack and must fill it with the most valuable items. The problem often arises in resource allocation where the decision makers have to choose from a set of non-divisible projects or tasks under a fixed budget or time constraint, respectively.
>
> The most common problem being solved is the 0-1 knapsack problem, which restricts the number ${\displaystyle x_{i}}$ of copies of each kind of item to zero or one. Given a set of ${\displaystyle n}$ items numbered from 1 up to ${\displaystyle n}$, each with a weight ${\displaystyle w_{i}}$ and a value ${\displaystyle v_{i}}$, along with a maximum weight capacity ${\displaystyle W}$,
>
\begin{align}
\mbox{maximize } &\sum _{i=1}^{n}v_{i}x_{i}\\
\mbox{subject to,}\qquad\\ 
\sum _{i=1}^{n}w_{i}x_{i}&\leq W\\
x_{i}&\in \{0,1\}.
\end{align}
>Here $\displaystyle x_{i}$ represents the number of instances of item $\displaystyle i$ to include in the knapsack. Informally, the problem is to maximize the sum of the values of the items in the knapsack so that the sum of the weights is less than or equal to the knapsack's capacity.

A more explicit formulation is:

$\underline{\mbox{Sets & Indices}}$</br>
$\mathcal{I}$ - set of items, $i\in\mathcal{I}$

$\underline{\mbox{Parameters}}$</br>
$v_{i}$ - value of item $i$, $\forall i\in\mathcal{I}$.</br>
$w_{i}$ - capacity required to include item $i$, $\forall i\in\mathcal{I}$.</br>
$W$ - available capacity.</br>

$\underline{\mbox{Decision Variables}}$</br>
$x_{i}$ - 1, if the optimal solution includes item $i$; 0, otherwise.</br>


\begin{align}
\mbox{maximize } &\sum _{i\in\mathcal{I}}v_{i}x_{i}\\
\mbox{subject to,}\qquad\\ 
\sum _{i\in\mathcal{I}}w_{i}x_{i}&\leq W\\
x_{i}&\in \{0,1\}, \forall i \in \mathcal{I}.
\end{align}

The following code block imports Gurobi.

In [ ]:
from gurobipy import *

The following code block randomly creates the required sets and parameters for a number of items specified by $n$. 

In [ ]:
np.random.seed(0)

n = 30
I = {i for i in range(1, n+1)}
w = {i: np.random.uniform(low = 1, high = 10) for i in I}
v = {i: np.random.uniform(low = 1, high = 10) for i in I}
W = n*3

The following code block defines a Gurobi `Model` object for our problem. 

In [ ]:
ks_model = Model('Knapsack')

The following code block defines our decision variables.

In [ ]:
x = ks_model.addVars(I, 
                     vtype = GRB.BINARY, 
                     name = 'x')

The following code block defines our objective function.

$$\mbox{maximize } \sum _{i\in\mathcal{I}}v_{i}x_{i}$$

In [ ]:
total_value = LinExpr()
for i in I:
    total_value.add(v[i]*x[i])
ks_model.setObjective(total_value, sense = GRB.MAXIMIZE)

The following code block defines our single **SET** of constraints.

$$\sum _{i\in\mathcal{I}}w_{i}x_{i}\leq W$$

In [ ]:
total_weight = LinExpr()
for i in I:
    total_weight.add(w[i]*x[i])
    
ks_model.addConstr(total_weight <= W)

The following code block optimizes our model.

In [ ]:
ks_model.optimize()

The following code block prints 10 items included in the optimal solution.

In [ ]:
items_to_include = [i for i in I if x[i].x > 0]
items_to_include[:10]        

The following code block prints additional details regarding the solution.

In [ ]:
print(f'Optimal solution includes {len(items_to_include)} of {n} items for a total value of {ks_model.ObjVal}')

capacity_required = np.round(sum([w[i] for i in items_to_include]))

print(f'The optimal solution uses {capacity_required} of the available {W} units of capacity')

# How well could we do on our own?

The following code block creates a *value per unit capacity metric* and sorts the items by this metric.

In [ ]:
value_per_unit_capacity = {i: v[i]/w[i] for i in I}
item_order = [k for k, v in sorted(value_per_unit_capacity.items(), key = lambda item: item[1], reverse = True)]

The following code block demonstrates a simple *greedy* heuristic for the problem.

In [ ]:
available_capacity = W

heuristic_solution = []
heuristic_value = 0
for item in item_order:
    if w[item] < available_capacity:
        available_capacity -= w[item]
        heuristic_solution.append(item)
        heuristic_value += v[item]

The following code block prints additional details regarding the solution.

In [ ]:
print(f'Heuristic solution includes {len(heuristic_solution)} of {n} items for a total value of {heuristic_value}')

capacity_required = np.round(sum([w[i] for i in heuristic_solution]))

print(f'The heuristic solution uses {capacity_required} of the available {W} units of capacity')

## A Larger instance

In [ ]:
data = pd.read_csv('sample.csv')
data = data.set_index('item')
I = set(data.index.to_list())
v = data['v'].to_dict()
w = data['w'].to_dict()
W = 49877

In [ ]:
ks_model = Model('Knapsack')

x = ks_model.addVars(I, 
                     vtype = GRB.BINARY, 
                     name = 'x')

total_value = LinExpr()
for i in I:
    total_value.add(v[i]*x[i])
ks_model.setObjective(total_value, sense = GRB.MAXIMIZE)

total_weight = LinExpr()
for i in I:
    total_weight.add(w[i]*x[i])
    
ks_model.addConstr(total_weight <= W)

ks_model.optimize()

In [ ]:
print(f'Optimal solution includes {len(items_to_include)} of {n} items for a total value of {ks_model.ObjVal}')

capacity_required = np.round(sum([w[i] for i in items_to_include]))

print(f'The optimal solution uses {capacity_required} of the available {W} units of capacity')

In [ ]:
value_per_unit_capacity = {i: v[i]/w[i] for i in I}
item_order = [k for k, v in sorted(value_per_unit_capacity.items(), key = lambda item: item[1], reverse = True)]

available_capacity = W

heuristic_solution = []
heuristic_value = 0
for item in item_order:
    if w[item] < available_capacity:
        available_capacity -= w[item]
        heuristic_solution.append(item)
        heuristic_value += v[item]

In [ ]:
print(f'Heuristic solution includes {len(heuristic_solution)} of {n} items for a total value of {heuristic_value}')

capacity_required = np.round(sum([w[i] for i in heuristic_solution]))

print(f'The optimal solution uses {capacity_required} of the available {W} units of capacity')

# A *Harder* Problem

Gurobi seems to be able to handle the Knapsack problem with ease. Is this true for all problems? We will now see how Gurobi works on the Traveling Salesman Problem (TSP). From https://en.wikipedia.org/wiki/Travelling_salesman_problem (accessed 2/17/2021):

> The travelling salesman problem (also called the traveling salesperson problem or TSP) asks the following question: "Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city exactly once and returns to the origin city?" It is an NP-hard problem in combinatorial optimization, important in theoretical computer science and operations research.

Essentially, if we were to find an algorithm that can determine a **Provably Optimal** solution to the TSP that has a runtime that can be bound by a polynomial function of the problem input size, i.e., we can find an algorithm $f$ that has runtime $O(f(n)) \leq n^{p}$, for some value of $p$, we would be able to settle the debate regarding the classes **P** and **NP** (and win a million dollars, see https://www.claymath.org/millennium-problems/p-vs-np-problem).

The following chart shows how the number of years to check all of the solutions to a TSP problem on the worlds fastest supercomputer varies as a function of the problem size.

In [ ]:
import math
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

stats = []
for n in range(2, 35):

    possible_solutions = math.factorial(n)
    computations_per_second = 200000000000000000
    seconds_required = possible_solutions//computations_per_second
    years_required = seconds_required//(60*60*24*365)
    stats.append([n, max(1, years_required)])
    
stats = pd.DataFrame(stats, columns = ['n', 'years'])

fig, ax = plt.subplots(1, 1, figsize = (8, 5))

sns.lineplot(
    x = 'n',
    y = 'years',
    data = stats,
)
ax.set_yscale('log')

plt.show()

Estimates suggest a UPS driver makes roughly 120 drops a day. How many years would be required to check all possible solutions?

In [ ]:
n = 120
possible_solutions = math.factorial(n)
computations_per_second = 200000000000000000
seconds_required = possible_solutions//computations_per_second
years_required = seconds_required//(60*60*24*365)
years_required

Let's see how Gurobi performs on the UPS-sized problem. Note: code modified from that available at https://www.gurobi.com/documentation/9.1/examples/tsp_py.html 

In [ ]:
import math
import random
from itertools import combinations
import gurobipy as gp
from gurobipy import GRB


# Callback - use lazy constraints to eliminate sub-tours
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                                if vals[i, j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < n:
            # add subtour elimination constr. for every pair of cities in tour
            model.cbLazy(gp.quicksum(model._vars[i, j]
                                     for i, j in combinations(tour, 2))
                         <= len(tour)-1)


# Given a tuplelist of edges, find the shortest subtour

def subtour(edges):
    unvisited = list(range(n))
    cycle = range(n+1)  # initial length has 1 more city
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle

random.seed(1)
points = [(random.randint(0, 100), random.randint(0, 100)) for i in range(n)]

# Dictionary of Euclidean distance between each pair of points

dist = {(i, j):
        math.sqrt(sum((points[i][k]-points[j][k])**2 for k in range(2)))
        for i in range(n) for j in range(i)}

m = gp.Model()

# Create variables

vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='e')
for i, j in vars.keys():
    vars[j, i] = vars[i, j]  # edge in opposite direction

# Add degree-2 constraint

m.addConstrs(vars.sum(i, '*') == 2 for i in range(n))

# Optimize model

m._vars = vars
m.Params.lazyConstraints = 1
m.Params.TimeLimit = 60
m.optimize(subtourelim)

vals = m.getAttr('x', vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

tour = subtour(selected)
assert len(tour) == n

print('')
print('Optimal tour: %s' % str(tour))
print('Optimal cost: %g' % m.objVal)
print('')